# 트립 어드바이저 MySQL과 연동


In [3]:
import scrapy
import requests as rq
import time
import pymysql #mysql import 모듈
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as BS

In [4]:
path = 'C:\ChromeDriver\chromedriver.exe'
PAUSE_TIME = 1.0

In [5]:
driver = webdriver.Chrome(path)
time.sleep(PAUSE_TIME)

In [6]:
url = "https://www.tripadvisor.co.kr/Restaurants-g297886-Daegu.html"

In [7]:
driver.get(url)
time.sleep(1)

In [8]:
text = driver.page_source
soup = BS(text, 'html.parser')

In [9]:
# 지역이 어딘지 확인
head = soup.find('h1', {'id':'HEADING'})
head_n = head.text.strip()
print(" < " + head_n + " > ")

 < 대구 음식점 > 


In [10]:
revs = soup.findAll('div', {'class':'title'})
print(" < " + head_n + " > ")

 < 대구 음식점 > 


In [11]:
# 30위까지 식당 이름 및 링크 가져오기
index = 1
alink=[]
for rev in revs:
    data = rev.text.strip()
    link = rev
    if data == "받은 메시지함":
        continue
    elif data == "추가 보기":
        continue
    else:
        print("%d 위 :" %index, data)
        alink.append("https://www.tripadvisor.co.kr/" + rev.a['href'])
        print("링크 : https://www.tripadvisor.co.kr/" + rev.a['href'] + "\n")
        
        index += 1

1 위 : 하이 타이
링크 : https://www.tripadvisor.co.kr//Restaurant_Review-g297886-d7943036-Reviews-Hi_Thai-Daegu.html

2 위 : 바라지레스트랑
링크 : https://www.tripadvisor.co.kr//Restaurant_Review-g297886-d7010929-Reviews-Balaji_Restaurant-Daegu.html

3 위 : Deira Restaurant
링크 : https://www.tripadvisor.co.kr//Restaurant_Review-g297886-d10218361-Reviews-Deira_Restaurant-Daegu.html

4 위 : 중앙떡볶이
링크 : https://www.tripadvisor.co.kr//Restaurant_Review-g297886-d8055192-Reviews-Jungang_Tteokbokki-Daegu.html

5 위 : 뉴살라딘
링크 : https://www.tripadvisor.co.kr//Restaurant_Review-g297886-d7265417-Reviews-New_Saladdin-Daegu.html

6 위 : 리틀 이탈리아
링크 : https://www.tripadvisor.co.kr//Restaurant_Review-g297886-d6732879-Reviews-Little_Italia-Daegu.html

7 위 : 금곡삼계탕
링크 : https://www.tripadvisor.co.kr//Restaurant_Review-g297886-d4034370-Reviews-Geumgok_Samgyaetang-Daegu.html

8 위 : 황금돼지
링크 : https://www.tripadvisor.co.kr//Restaurant_Review-g297886-d9015520-Reviews-Gold_Pig-Daegu.html

9 위 : 빈플러스
링크 : https://www.tripadvisor.co.k

In [12]:
# 컴퓨터 MySQL DB연결
conn = pymysql.connect(host = 'localhost', user = 'beans', password = '', db = 'trivago_db', charset = 'utf8')

In [13]:
curs = conn.cursor()

In [14]:
try:
    ds = 'drop database trivago_db'
    curs.execute(ds)
    cs = 'create schema trivago_db'
    curs.execute(cs)
except InterfaceError:
    cs = 'create schema trivago_db'
    curs.execute(cs)


In [15]:
estring = ['안녕히', 'Hello', 'Elly Jin', 'Jihwaja','일동', '고맙', '감사드', '죄송합니다', '에런알렌&권혜진', '고객님께', '찾아주셔서', '타볼로24팀', '남겨주신', '박기영 배상', 'Nabezo Shinjuku-Sanchome Branch']
#댓글의 댓글까지 긁어져서 방지하고자 분류설정

for link in alink:
    driver.get(link) #음식점 링크로 이동
    time.sleep(PAUSE_TIME)
    try:
        name = driver.find_element_by_xpath('//*[@id="HEADING"]').text
    except:
        name = driver.find_element_by_xpath('//*[@id="taplc_resp_rr_top_info_rr_resp_0"]/div/div[1]/h1').text #음식점 이름
        
    print("<", name,">")
    
    #이름 전처리
    if name[0] in [0-9]:
        n = '_' + name
    else:
        pass
    
    if ' ' in name:
        nam = name.replace(', ', '_')
        nam = nam.replace(' ', '_')
        nam = nam.replace('-', '_')
    else:
        nam = name
    
    
    
    ct = 'create table trivago_db.' + str(nam) + ' (id varchar(20) not null, title varchar(150) not null, subtext text not null, primary key(id))'
    sql = 'insert into  trivago_db.' + str(nam) + ' (id, title, subtext) values (%s, %s, %s)'
    curs.execute(ct) #DB 테이블 음식점별로 생성
    driver.find_element_by_class_name('noQuotes').click()
    time.sleep(PAUSE_TIME)
    #들어갔을 때 맨 처음 리뷰 클릭
    try:
        num = driver.find_element_by_xpath('//*[@id="taplc_location_reviews_list_sur_0"]/div/p/b[1]')
        n = int(num.text)
        if (n % 5) != 0 :
            n = int(n / 5) + 1
        else:
            n = int(n / 5)
    except:
        n = 1
        pass
    

    
    for i in range(n):
        text = driver.page_source
        soup = BS(text, 'html.parser')
        time.sleep(PAUSE_TIME)
        id = soup.findAll('span', 'expand_inline scrname')
        rtl = driver.find_elements_by_class_name('noQuotes') #리뷰 소제목
        stl = driver.find_elements_by_class_name('entry') #리뷰 세부내용
        
        for a in stl:
            try:
                for word in estring:
                    if word in a.text:
                        stl.remove(a)
            except:
                pass
        
        for i in range(len(rtl)):
                try:
                    print('ID : ' + id[i].text)
                    print("title :", rtl[i].text.strip())
                    print("subtext :", stl[i].text.strip() +'\n')
                    curs.execute(sql, (id[i].text, rtl[i].text.strip(), stl[i].text.strip()))
                    conn.commit()
                except:
                    pass
                
        
        try:
            driver.find_element_by_xpath('//*[@id="taplc_location_reviews_list_sur_0"]/div/div[11]/div/div/a[2]/div').click()
            time.sleep(PAUSE_TIME)
        except:
            pass
        
    print('='*50)

conn.close()
                
        
        

< 하이 타이 >
ID : GlobeTrotting675654
title : 완전 맛난 타이요리~~^^
subtext : 방콕이 그리울때 가는식당..
태국요리를 병형없이해서 넘좋아요.
뭘 먹어도 맘에 들고 맛있는 집.
특히 쏨땀, 모닝글로리 ,똠양꿍,바질덮밥은 즐겨먹는요리 예요~~~^^

ID : mys0ng
title : 맛있는데 양이 적어요
subtext : 맛있어서 몇 번 갔는데 둘이서 2개 시키면 늘 부족했어요. 3개 시켜야 맞나봅니다. 분위기도 좋고 친절해서 좋습니다. 하지만 양에 비해 가격이 비싸다는 생각이 들어요. 가격은 그대로 하고 양을 좀 늘려주면 자주 갈 것 같아요!

ID : exercise11
title : 대구에서 단연 타이음식은 이곳
subtext : 작은 식당.
그다지 빠르지 않은 음식 서빙 속도.
슬로우 푸드
전형적 현지식
대구에서 타이음식은 단연 이곳.
맛있음. 맥주랑 함께여야

ID : Destination802878
title : 그냥 보통입니다
subtext : 가격이 매우 비싸요
맛도 그냥 그렇구요 한국인이 운영하지만 태국음식 맛은 많이 안나요
똠양꿍은 그럭저럭 먹을만 하지만 다른 음식을 딱히 추천하지는 않습니다
특히 모닝글로리 가격이 말도 안되게 비쌉니다.

ID : veronica0215
title : So..so.
subtext : 맛집이라기 보다. 그냥 친구와 가볍게 갈수 있는 식당입니다.
그러나 2번 가고 싶지는 않다는.
음식맛이 너무 인스턴트음식 같다는.

ID : GlobeTrotting675654
title : 완전 맛난 타이요리~~^^
subtext : 방콕이 그리울때 가는식당..
태국요리를 병형없이해서 넘좋아요.
뭘 먹어도 맘에 들고 맛있는 집.
특히 쏨땀, 모닝글로리 ,똠양꿍,바질덮밥은 즐겨먹는요리 예요~~~^^

ID : mys0ng
title : 맛있는데 양이 적어요
subtext : 맛있어서 몇 번 갔는데 둘이서 2개 시키면 늘 부족했어요. 3개 시켜야 맞나봅니다. 분위기도 좋고 친절해서 좋습니다.

title : 개존맛탱
subtext : 여기 대구 동성로에서 제일 유명한 떡볶이 집임 ㅜㅠㅜㅜㅜㅜㅜ다들 드세요 백번드세요 쌀떡 최고입니다 그리고 납작만두 지리죠 ㅜㅜㅜㅜㅜㅜㅜㅜ캬

ID : JangHunL1
title : 안맵고 맛있어요
subtext : 떡볶이 매운걸 잘 못먹는데 여기는 안맵고 맛있어요 가래떡이라서 쫀뜩쫀뜩하게 맛잇습니다. 납작만두도 맛나요 추천해요!

ID : You_have_been_here
title : 왜 맛집인지 모르겠다
subtext : 삼대천왕의 백종원 아저씨를 믿고 갔다. 번화가라고 하기엔 뭔가 못미쳤지만... 그나마 북적거리는 곳 쯤에 위치했다.

우리가 갔을 땐 다 팔고 새로 만들어야 해서 30분이 걸릴 거라고 했다. 동의하고 가게 들어갔더니 번호표를 줬다. 잠깐 주변 둘러보고 오면 안되냐 물어봤더니 번호표를 받는 순간 나갈 수가 없단다. 나중에 자리 없을 수 있다고, 이 점은 그럴 수도 있지라고 생각했다. 근데 추운데 문을 안 닫아줘서 기다리는 내내 발 동동 굴렸다.

결국 30분 뒤 받은 떡볶이는 너무 평범했다. 크 ㄴ가래떡으로 한 떡볶이란 점만 빼면 뭐 그리 줄서고 먹는 음식인가 싶다. 우리처럼 기대심에 여행온 사람들이 이렇게 많을까, 설마 먹어본 사람들이 줄서서 먹는 건가 라는 의구심이 들 정도였다.

양념은 카레가루를 좀 넣었는지 카레맛이 조금 나는 정도.

결론은 이렇게 기다려서 먹을 음식이 아니라는 것이다.

ID : Vacation654152
title : 친구랑 간단한 점심
subtext : 늘 번호표 뽑고 기다려야 하는곳 ^^
그래도 기다린 보람이 있는곳
납작만두에 떡볶이 국물에 찍어 먹으면 정말 맛있습니다.
김밥도 괜찮구요.

ID : Camper825056
title : 맛
subtext : 줄이 끊이지않는곳이라 방문
그래도 요즘은 예전만큼 실시간 줄서있지는 않다.
떡만 먹어도 맛있고
납작만두와 떡뽁이국물조합이 아주 맛나다.
그러나
계속 단골할 맛은 아니다

ID : Voyage746812
title 

subtext : 떡볶이 매운걸 잘 못먹는데 여기는 안맵고 맛있어요 가래떡이라서 쫀뜩쫀뜩하게 맛잇습니다. 납작만두도 맛나요 추천해요!

ID : You_have_been_here
title : 왜 맛집인지 모르겠다
subtext : 삼대천왕의 백종원 아저씨를 믿고 갔다. 번화가라고 하기엔 뭔가 못미쳤지만... 그나마 북적거리는 곳 쯤에 위치했다.

우리가 갔을 땐 다 팔고 새로 만들어야 해서 30분이 걸릴 거라고 했다. 동의하고 가게 들어갔더니 번호표를 줬다. 잠깐 주변 둘러보고 오면 안되냐 물어봤더니 번호표를 받는 순간 나갈 수가 없단다. 나중에 자리 없을 수 있다고, 이 점은 그럴 수도 있지라고 생각했다. 근데 추운데 문을 안 닫아줘서 기다리는 내내 발 동동 굴렸다.

결국 30분 뒤 받은 떡볶이는 너무 평범했다. 크 ㄴ가래떡으로 한 떡볶이란 점만 빼면 뭐 그리 줄서고 먹는 음식인가 싶다. 우리처럼 기대심에 여행온 사람들이 이렇게 많을까, 설마 먹어본 사람들이 줄서서 먹는 건가 라는 의구심이 들 정도였다.

양념은 카레가루를 좀 넣었는지 카레맛이 조금 나는 정도.

결론은 이렇게 기다려서 먹을 음식이 아니라는 것이다.

ID : Vacation654152
title : 친구랑 간단한 점심
subtext : 늘 번호표 뽑고 기다려야 하는곳 ^^
그래도 기다린 보람이 있는곳
납작만두에 떡볶이 국물에 찍어 먹으면 정말 맛있습니다.
김밥도 괜찮구요.

ID : Camper825056
title : 맛
subtext : 줄이 끊이지않는곳이라 방문
그래도 요즘은 예전만큼 실시간 줄서있지는 않다.
떡만 먹어도 맛있고
납작만두와 떡뽁이국물조합이 아주 맛나다.
그러나
계속 단골할 맛은 아니다

ID : Voyage746812
title : 개존맛탱
subtext : 여기 대구 동성로에서 제일 유명한 떡볶이 집임 ㅜㅠㅜㅜㅜㅜㅜ다들 드세요 백번드세요 쌀떡 최고입니다 그리고 납작만두 지리죠 ㅜㅜㅜㅜㅜㅜㅜㅜ캬

ID : JangHunL1
title : 안맵고 맛있어요
subtex

ID : 수림 정
title : 진짜 맛있어요~
subtext : 진짜 맛있어요~! 무르그마커니랑 탄두리치킨, 갈릭난 먹었는데 다시 오고싶어요ㅎㅎ 디저트에 짜이랑 라씨도 맛있어용♡

< 리틀 이탈리아 >
ID : goodluck1004
title : 파스타 맛집
subtext : 대구 동성로에 정통 이탈리아식 파스타집이에요.
깔끔한 실내인테리어 만큼 음식도 아주 깔끔하게 나와요.
정말 맛잇습니다. 꼭 들러보세요.

ID : kyungs00
title : 괜찮은 이탈리아 식당집
subtext : 퓨전 이탈리아 음식집인데 괜찮아요.
다만 가격이 비싼데 양이 적다고 느껴졌고 전문점이란 생각은 크게 들지 않았습니다. 그거빼곤 만족스러워요.

ID : Trek771938
title : 굿입니다
subtext : 가족여행 하던중 방문한 곳입니다.
매우 만족스럽고 분위기도 좋은곳이라서 입소문내려고 이렇게 리뷰 작성중입니다~
가족단위로 와도 좋은곳이니 추천해요

ID : leoi811
title : 작은 리틀 이탈리아
subtext : 작은 리틀 이탈리아는 대구 동성로에 위치하고 있어요 올리브 파스타를 추천해요 가격도 저럼하고 좋아요

ID : nina22077
title : 좋아요
subtext : 오래된 집이라 '추억의 그집'인 느낌.
맛도 괜찮고 친구한테 한턱 내기에도 맛이나 분위기 등등 적당해서 좋아했어요.
곧 없어진다고 해서 별을 뺐어요. 아쉽네요

ID : Munyeong
title : 맛있어요
subtext : 파스타가 맛있기로 유명한 곳이에요. 주로 식사시간에 방문하신다면 미리 예약을 하시는 것도 좋을 것 같아요. 소문만큼 맛있었습니다.

ID : sangukh2015
title : 정겨운곳..
subtext : 대구 동성로에 위치하여 접근성이 좋고
정겹고 친절하고 편안한 분위기속에 좋은 음식 그리고 음악 모든것 좋았습니다 호텔 노보텔과 엘디스가 가깝습니다.

ID : xururuca319
title : 맛있습니다
subtext : 테이블이 많지 않은 편이고 

subtext : 요기 대구에서 젤 맛잇는 시카고피자팔아용
식어도 맛잇어요
대신 웨이팅이 길어서 ㅠㅠ너무 아쉬워요

ID : Climber677180
title : 분위가와 가성비 굿
subtext : 아는동생 만나러 저녁을 여기서 해결했어요.
금액도 많이 나어지않았구 양도 푸짐해서 둘이서 먹기 부족함이 없었어요!! 평일 주말 둘다 갔는데 주말에 가면 약간의 웨이팅이 있을 수 있어요. 시내 (클럽골목) 근처라 사람들이 많고요

ID : Camper825056
title : 그냥저냥
subtext : 20대초반들이 한 번쯤 갈만 한 곳
양이 많다.
추천 음료 별로다.
사람이 너무 많다.
어중간한 시간 추천.
두번이상 가기는 별로
다른곳추천

ID : Expedition732467
title : 빈플러스
subtext : 양 엄청많고! 사람도많고
오픈시간이랑 거의맞물려서 갔는데 좀만늦었음 기다릴뻔
맛있고 메뉴도 다양하고! 친구추천받아서 온건데 좋았어용~~~

ID : lovelyjh0714
title : 대구에서 시카고 피자 먹기
subtext : 완전히 어메리칸 스타일 맛은 아니지만
풍부한 치즈맛을 느낄 수 있는 곳이예요.
다만 많이 먹으면 조금 느끼 할 수 있어서
여럿이 가서 먹으면 좋을 곳

ID : RH220
title : 시카고피자 ㅠㅠ
subtext : 요기 대구에서 젤 맛잇는 시카고피자팔아용
식어도 맛잇어요
대신 웨이팅이 길어서 ㅠㅠ너무 아쉬워요

ID : Climber677180
title : 분위가와 가성비 굿
subtext : 아는동생 만나러 저녁을 여기서 해결했어요.
금액도 많이 나어지않았구 양도 푸짐해서 둘이서 먹기 부족함이 없었어요!! 평일 주말 둘다 갔는데 주말에 가면 약간의 웨이팅이 있을 수 있어요. 시내 (클럽골목) 근처라 사람들이 많고요

ID : Camper825056
title : 그냥저냥
subtext : 20대초반들이 한 번쯤 갈만 한 곳
양이 많다.
추천 음료 별로다.
사람이 너무 많다.
어중간한 시간 추천.
두번이상 가기는

ID : GoPlaces808329
title : 비싸지만 맛있다.
subtext : 솔직히 푸팟퐁커리에 들어가는 꽃게는 해가 가면 갈수록 작아지는듯;;;;
그래도 대구에 이정도 맛을 내는 곳이 잘 없어서 가게되는데 게 크기가 아쉽다.
가격대도 좀 있어서 둘이가도 5만원은 그냥 나오는듯...

ID : przza1009
title : 팬아시아
subtext : 팬아시아는 태국등 여러아시아의 음식을 맛볼수있는곳입니다~ 가셔서 푸팟퐁커리 꼭 드셔보세요~~

ID : JinsilC7
title : 별로임
subtext : 음식이 짜고 별로 였어요. 사람이 많은 것도 아니었는데 주문도 잘 못 알아듣고, 뭔가 휑하니 분위기도 별로 였어요

ID : lsh890131
title : 가끔씩 신선한 메뉴가 필요할때 딱!
subtext : 파스타 피자 등등 식상한 외식메뉴에ㅡ질렸을때 딱!
크리스마스때 6명 모임장소로 선택했었는데
예약석도 딱 잘준비해주시고,, 메뉴도 한국인입맛에 맞고 좋았어요!

ID : soominc18
title : 맛나여^^
subtext : 음식은 맛있고 깔끔한것 같아요
모히또 시키면 버스켓에 담아주는데
이게 재밌고 맜있어요!!!!!

ID : nina22077
title : 붐비는 시간 피하세요
subtext : 대기시간이 길어요. 지금있는 건물 맞은편에 있을때가 더 좋았던 것 같지만 여전히 맛있음. 현대백화점 지하에도 있는데 메뉴가 완전 다름

ID : xururuca319
title : 괜찮습니다
subtext : 싱가폴 및 태국 음식점으로칠리크랩이 가장 대표적인 메뉴입니다. 팟타이가 조금 단 편이지만 맛있습니다. 모히또 음료도 맛있는 편입니다.

ID : BonSports
title : 독특한 음식을 원한다면
subtext : 동남아 음식이 특기인 곳으로, 맛도 괜찮다. 하지만 대기가 많았고, 생각보다 가격이 비쌌다. 독특한 음식을 원한다면 가볼만하다.

ID : Dreamer807669
title : 특별하지만 맛있진않았어요
subtext : 

title : 숨은맛집입니다
subtext : 지나가다 인테리어가 예뻐서 들렸는데 오오 맛있더군요 직접 반죽도 만드신다했던 치킨바오와 아보카도샐러드 건강해질것같은 맛있게잘먹었어요 재방문의사있음~!

ID : Euna L
title : 건강한 한끼
subtext : 편안한 분위기에서 건강한 식사를 할 수 있어서 좋았습니다.
아보카도 샐러드와 시금치 케일 스프를 먹었는데 정말정말 맛있게 잘 먹었습니다 !

ID : Jihyung S
title : Fresh healthy food and Warm cozy place
subtext : 건강한 음식을 아주 맛있게 내어주는 식당입니다. 특히 겨울에 스프 먹고 싶을 때 항상 생각나는 집이예요. 그리고 바오가 특이해요! 치킨 바오랑 매콤귀리샐러드 그리고 스프까지 먹으면 너무 많이 배가 부르긴 하지만 정말 맛있어요!

< 한옥집 대구점 >
ID : BackPacker762337
title : 김치찜 맛집
subtext : 10번도 넘게 갔던거 같아요 ㅋㅋ 싼값에 배부르게 먹고옵니다
김치찜 1 + 김치찌개 1 시키면 돼요~ 밥은 본인이 리필해서 계속 드실수있어요
그리고 여기 집 콜라가 병콜라인데 ㅋㅋ 맛있어요

ID : StephanieK3565
title : 가성비 좋은 곳
subtext : 김치찜과 김치찌개만 사람 수대로 시키면 공기밥과 라면사리는 무한리필이라 가성비 좋은 곳. 단점은 먹고 나오면 온 몸과 옷에 김치냄새가 찌들어 중요한 약속을 잡을 수가 없다는 정도?

ID : Aegyeong K
title : 김치찌 맛집!
subtext : 김치찜 맛집입니다 삶은 김치와 돼지고기를 같이 곁들어 먹으면 아주 맛있습니다 주차는 조금 불편합니다

ID : 민수 김
title : 김치찜이 맛있는 곳
subtext : 김치찜으로 유명한 곳으로 라면 사리를 무제한으로 먹을 수 있다.
김치와 함께 돼지고기가 함께 나오는데, 돼지고기가 부드럽고 맛있음
가격도 나름 저렴한편

ID : hith6
title : 만족
subtext : 6천

ID : Karl-Styner
title : 동성로 전원돈까스
subtext : 동성로 있는 유명 돈까스집.
오랜 기간동안 대구 사람들의 입맛에 맞는 맛집으로 자리잡아 왔고
노하우가 쌓여서 빠르게 음식을 만들 수도 있다.
내부 인테러이더 괜찮은 편.

ID : kj3378
title : 추억에 음식이에요
subtext : 고등학생때부터 먹었던 돈까스집입니다
소스맛이 아주 일품이에요
일본식 돈까스도 좋지만
가끔 이런돈까스가 그리울때가 많아요
꼭한번가보세요~

ID : 민수 김
title : 전통있는 경양식 돈가스
subtext : 정말 오래된 경양식 돈가스 집으로 빠른 서빙이 가능하고 가격도 저렴함
일식 돈가스와는 또 다른 맛이 느껴져요

ID : leoi811
title : 대구에 유명한 명물
subtext : 대구에 유명한 명물 전원돈까스는 학창시절때 부터 다니던 단골 돈까스 집인데 맛도 깔끔하구요

ID : Storyteller807389
title : 가족과 같이한 저녁
subtext : 가족과 시내 볼일보러 갔다 우연히 방문한집입니다 옛날 생각나게하는 돈까스집 분위기에 친절하고 빠른 서비스가 좋았습니다
푸짐한 돈까스는 잘 튀겨져 매우 맛있었습니다
곱배기를 주문하면 고기가 두덩어리가 나오고 후식으로주는 콜라 한잔이 인상적이었습니다

ID : Pioneer688994
title : 추억의장소
subtext : 참 20년전에 먹고 오랫만에 먹는데도 좋네요 추억의장소라 ㅇ+1점 더 드립니다 대구오시면 한번 드세요 가격도 부담 스럽지않습니다

ID : _E5396OM
title : 추억의 돈까스 가게
subtext : 어릴 적 외식 메뉴 우선 순위 1위는 돈까스
부모님 따라 간 전원돈까스. 돈까스를 먹는 날은 행복에 겨웠던 곳.
지금도 간혹 추억의 맛을 찾아 간다.
아들이 크면 데리고 가고 싶은 곳.

ID : Yongtae J
title : 대구에서 가장 유명한 돈까스집
subtext : 대구에서 아주 오래된 경양식 돈까스집입니다. 한장소에 계속 있었으며 제가

InternalError: (1050, "Table '뉴살라딘' already exists")

## 일회성 시험본

In [ ]:
driver.get(link) #음식점 링크로 이동
time.sleep(PAUSE_TIME)
try:
    name = driver.find_element_by_xpath('//*[@id="HEADING"]').text
except:
    name = driver.find_element_by_xpath('//*[@id="taplc_resp_rr_top_info_rr_resp_0"]/div/div[1]/h1').text #음식점 이름
    
print("<", name,">")
    
#이름 전처리
if name[0] in [0-9]:
    n = '_' + name
else:
    pass
    
if ' ' in name:
    nam = name.replace(', ', '_')
    nam = nam.replace(' ', '_')
    nam = nam.replace('-', '_')
else:
    pass
    
    
    
ct = 'create table trivago_db.' + str(n) + ' (id varchar(20) not null, title varchar(150) not null, subtext text not null, primary key(id))'
sql = 'insert into trivago_db.' + str(n) + ' (id, title, subtext) values (%s, %s, %s)'
curs.execute(ct) #DB 테이블 음식점별로 생성
driver.find_element_by_class_name('noQuotes').click()
time.sleep(PAUSE_TIME)
#들어갔을 때 맨 처음 리뷰 클릭
try:
    num = driver.find_element_by_xpath('//*[@id="taplc_location_reviews_list_sur_0"]/div/p/b[1]')
    n = int(num.text)
    if (n % 5) != 0 :
        n = int(n / 5) + 1
    else:
        n = int(n / 5)
except:
    n = 1
    pass
    

    
for i in range(n):
    text = driver.page_source
    soup = BS(text, 'html.parser')
    time.sleep(PAUSE_TIME)
    id = soup.findAll('span', 'expand_inline scrname')
    rtl = driver.find_elements_by_class_name('noQuotes') #리뷰 소제목
    stl = driver.find_elements_by_class_name('entry') #리뷰 세부내용
        
    for a in stl:
        try:
            for word in estring:
                if word in a.text:
                    stl.remove(a)
        except:
            pass
        
    for i in range(len(rtl)):
            try:
                print('ID : ' + id[i].text)
                print("title :", rtl[i].text.strip())
                print("subtext :", stl[i].text.strip() +'\n')
                curs.execute(sql, (id[i].text, rtl[i].text.strip(), stl[i].text.strip()))
                conn.commit()
            except:
                continue
                
        
    try:
        driver.find_element_by_xpath('//*[@id="taplc_location_reviews_list_sur_0"]/div/div[11]/div/div/a[2]/div').click()
        time.sleep(PAUSE_TIME)
    except:
        pass
        
print('='*50)

conn.close()